In [1]:
import pymongo
import datetime
import pytz
import numpy as np

def timestamp2datetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp,pytz.timezone("Europe/Warsaw"))


In [2]:
conn_backup = pymongo.MongoClient(host='localhost', port=27020, username='xflats',
                                  password='xflats', authSource='admin', authMechanism='SCRAM-SHA-256',
                                  serverSelectionTimeoutMS=5000)
db_bck = conn_backup['OFFERS']['Otodom_backup']

In [3]:
conn_org = pymongo.MongoClient(host='localhost', port=27017, username='xflats',
                                  password='xflats', authSource='admin', authMechanism='SCRAM-SHA-256',
                                  serverSelectionTimeoutMS=5000)
db_org = conn_org['OFFERS']['otodom']

In [4]:
project = {
    '_id':1,
    'name':1,
    'location':1,
    'flat_size':1,
    'rooms':1,
    'floor':1,
    'price':1,
    'tracking_id':1,
    'url':1,
    'producer_name':1,
    'main_url':1,
    'price_m2':1,
    'market':1,
    'number_of_floors':1,
    'floor_attic':1,
    'floor_basement':1,
    'building_type':1,
    'building_material':1,
    'widows_type':1,
    'heating_type':1,
    'year_of_building':1,
    'finishing_stage':1,
    'rent_price':1,
    'property_form':1,
    'available_from':1,
    'description':1,
    'additional_info':1,
    'GC_latitude':1,
    'GC_longitude':1,
    'GC_boundingbox':1,
    'GC_addr_house_number':1,
    'GC_addr_road':1,
    'GC_addr_neighbourhood':1,
    'GC_addr_suburb':1,
    'GC_addr_city':1,
    'GC_addr_county':1,
    'GC_addr_state':1,
    'GC_addr_postcode':1,
    'GC_addr_country':1,
    'GC_addr_country_code':1,
    'download_date':1
}

In [10]:
def transform_rec(rec):
    rec['_id'] = "oto_"+rec['_id']
    rec['flat_size'] = int(rec['flat_size']) if rec['flat_size'] is not None else None
    rec['rooms'] = int(rec['rooms']) if rec['rooms'] is not None else None
    rec['floor'] = int(rec['floor']) if rec['floor'] is not None else None
    rec['price'] = int(rec['price']) if rec['price'] is not None else None
    rec['tracking_id'] = int(rec['tracking_id']) if rec['tracking_id'] is not None else None
    rec['number_of_floors'] = int(rec['number_of_floors']) if rec['number_of_floors'] is not None else None
    rec['floor_attic'] = int(rec['floor_attic']) if rec['floor_attic'] is not None else None
    rec['floor_basement'] = int(rec['floor_basement']) if rec['floor_basement'] is not None else None
    rec['year_of_building'] = int(rec['year_of_building']) if rec['year_of_building'] is not None else None
    rec['download_date'] = timestamp2datetime(rec['download_date'])
    rec['backup'] = 1
    rec['producer_name'] = 'otodom'
    rec['main_url'] = 'https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/'

    return rec

In [ ]:
cnt_lim = 999999
cnt = 0
for row in db_bck.find({"price":{"$ne":np.NaN}},project):
    cnt += 1
    try:
        _tmp = transform_rec(row)
        db_org.insert_one(_tmp)    
    except BaseException as e:
        print("Exception, %s" % e) 
    if cnt >= cnt_lim:
        break

In [ ]:
rec = db_bck.find_one({},project)


In [ ]:
db_org.insert_one(rec)    